## Reinforcement Learning!

In lectia de astazi vom vorbi despre reinforcement learning si in acest notebook vom implementa un program care sa ajute o racheta cu 3 propulsoare (stanga, dreapta si mijloc) sa aterizeze pe o suprafata denivelata. Astfel, vom avea 4 actiuni posibile: sa nu facem nimic, sa activam propulsorul din stanga, cel din dreapta sau cel din mijloc.

Acest algoritm se bazeaza pe o abordare foarte intuitiva: vom stabili un set de rewarduri si pedepse pentru program, iar mereu cand face o actiune buna il rasplatim, iar cand face o actiune negativa il vom pedepsi. Astfel vom nota aceasta functie cu R(s), unde "s" este state-ul in care ne aflam. State-ul in acest caz va fi descris de urmatoarele valori: pozitiile pe axele x si y, vitezele pe aceste axe, unghiul rachetei, viteza unghiulara, daca piciorusul stang, respectiv drept al rachetei atinge pamantul sau nu.

De asemenea, putem defini o alta functie ajutatoare, Q(s, a) care ne da scorul pentru state-ul "s" in caz ca facem actiunea "a" daca dupa aceasta actiune ne vom comporta optim. Astfel, formula va fi Q(s, a) = R(s) + max(Q(s2, a')), unde "s" este stateul curent, "a" actiunea pe care vrem sa o facem, "s2" state-ul in care vom ajunge dupa ce facem actiunea "a", iar max(Q(s2, a')) reprezentand valoarea maxima posibila a functiei Q pentru stateul s2.

Pentru ca scopul nostru in reinforcement learning este sa realizam un task in cel mai scurt timp posibil, vom adauga un discount factor "γ" care va scade treptat scorul cu cat trece timpul. Pentru a exprima asta in mod matematic, vom da valori lui γ mai mici ca 1 si vom scrie functia Q ca o functie polinomiala in functie de γ: Q(s, a) = R(s) + γ * Q(s2, a') + γ^2 * Q(s3, a'') + γ^3 * Q(s4, a''') + ...
Daca dam factor comun γ, observam ca formula initiala devine Q(s, a) = R(s) + γ * max(Q(s2, a')).

Pare ca avem toate elementele necesare: pur si simplu trebuie la fiecare state sa vedem actiunea care ne aduce cel mai mare scor cu ajutorul functiei Q si actionam corespunzator. Problema noastra este urmatoarea: cum calculam functia Q? Momentam avem doar o exprimare a functiei Q in functie de alte valori ale functiei, asa ca nu prea ne ajuta. Astfel, ne putem gandi la o retea neuronala care sa aproximeze aceasta valoare. Daca antrenam una care sa ne mimeze functia Q?

Trainingul retelei va fi ceva in genul: vom tine minte un buffer de stateuri, actiuni, rewarduri etc si vom forma un dataset.
X-urile vor fi reprezentate de tuple-uri (s, a), iar Y-urile vor fi R(s) + γ * max(Q(s2, a')). Astfel, vom avea reteaua neuronala Q care va fi functia noastra si target_Q care va fi o retea identica auxiliara. La un pas de train vom antrena reteaua target_Q pe dataset, apoi vom transfera ce am invatat in reteaua noastra Q folosind soft update, adica preluam doar un procentaj din reteaua noua in cea de baza (Q) pentru a nu uita comportamente vechi.

Totusi, asta nu e tot. Pentru a incuraja modelul sa exploreze diferite approachuri, vom include un element de randomizare a actiunilor numit epsilon-greedy policy: la fiecare pas in care trebuie sa facem o actiune avem o sansa de a alege o actiune random si o sansa de a alege o actiune conform modelului nostru (epsilon). De asemenea, acest epsilon va scade treptat, odata ce ne perfectionam modelul.

#### Cum sa instalam biblioteciile

- pip install numpy
- pip install tensorflow
- pip install gym
- pip install box2d pygame
- pip install box2d-kengz
- pip install imageio ipython -> trebuie sa instalam ffmpeg (brew install ffmpeg pentru macos)

In [ ]:
# initial includem toate bibliotecile necesare
import time
import random
from collections import deque, namedtuple

# necesare pentru procesare si retele neuronale:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import MSE
from tensorflow.keras.optimizers import Adam

# necesare pentru simulator:
import gym

# necesare pentru video
import os
os.environ["IMAGEIO_FFMPEG_EXE"] = '/opt/homebrew/bin/ffmpeg' # pathul unde e instalat ffmpeg
import imageio
import IPython
import base64

In [ ]:
# dam setup simularii
env = gym.make('LunarLander-v2', render_mode='rgb_array')
env.reset()

state_size = env.observation_space.shape
num_actions = env.action_space.n

### Structura retelei

Initial ne-am gandi la o structura simpla: un input layer cu 12 neuroni (8 valori care descriu stateul, 4 valori care reprezinta one-hot encodingul unei actiuni), niste hidden layers si un output layer cu un neuron care sa ne dea valoarea Q(s, a).

Totusi, aceasta structura este putin ineficienta, pentru ca noi mereu vrem sa stim max(Q(s, a)), astfel vom apela modelul de 4 ori, ceea ce e foarte lent. Asadar, vom inlocui ultimul layer cu un neuron cu unul cu 4, pentru a avea mereu toate valorile la o singura apelare.

In [ ]:
# parametrii
MEMORY_SIZE = 100000 # ultimele 100k actiuni
GAMMA = 0.995 # discount factor
ALPHA = 1e-3 # learning rate
NUM_STEPS_FOR_UPDATE = 4 # modelul invata din 4 in 4 pasi de simulare

SEED = 0 # seed pentru initializare parametrilor random ai modelului
MINIBATCH_SIZE = 64
TAU = 1e-3 # procentaj pentru soft update
E_DECAY = 0.995 # rata la care scade ε pentru ε-greedy policy.
E_MIN = 0.01

In [ ]:
# retelele neuronale
q_network = Sequential([
    Input(shape=state_size),
    Dense(units=64, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=num_actions, activation='linear'), 
])

target_q_network = Sequential([
    Input(shape=state_size),
    Dense(units=64, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=num_actions, activation='linear'),
])

optimizer = Adam(learning_rate=ALPHA)

In [ ]:
def compute_loss(experiences, gamma, q_network, target_q_network):
    states, actions, rewards, next_states, done_vals = experiences
    
    # max target_Q(s,a)
    max_qsa = tf.reduce_max(target_q_network(next_states), axis=-1)
    
    # y = R daca se termina episodul de invatare, altfel y = R + γ max Q^(s,a).
    y_targets = rewards + gamma * max_qsa * (1 - done_vals)
    
    # luam valorile din modelul Q si le punem in aceeasi forma ca y_targets
    q_values = q_network(states)
    q_values = tf.gather_nd(q_values, tf.stack([tf.range(q_values.shape[0]), tf.cast(actions, tf.int32)], axis=1))
    
    loss = MSE(y_targets, q_values)
    return loss

In [ ]:
# soft update: luam p% din reteaua curenta si (100 - p)% din noua retea pentru stabilizare si pentru a nu uita anumite comportamente
def update_target_network(q_network, target_q_network):
    for target_weights, q_net_weights in zip(target_q_network.weights, q_network.weights):
        target_weights.assign(TAU * q_net_weights + (1.0 - TAU) * target_weights)

In [ ]:
# functia in care dam update retelei

@tf.function
def agent_learn(experiences, gamma):
    # calculam loss-ul
    with tf.GradientTape() as tape:
        loss = compute_loss(experiences, gamma, q_network, target_q_network)

    # luam derivatele din model
    gradients = tape.gradient(loss, q_network.trainable_variables)
    
    # dam update parametrilor retelei Q
    optimizer.apply_gradients(zip(gradients, q_network.trainable_variables))

    # dam update parametrilor retelei target_Q cu ajutorul soft update
    update_target_network(q_network, target_q_network)

In [ ]:
# functia care ne da actiunea cea mai buna bazata pe rezultatele date de retea si epsilon-greedy policy

def get_action(q_values, epsilon=0.0):
    if random.random() > epsilon:
        return np.argmax(q_values.numpy()[0])
    else:
        return random.choice(np.arange(4))

In [ ]:
# functie ajutatoare care ne spune daca este timpul sa dam update retelei target_Q

def check_update_conditions(t, num_steps_upd, memory_buffer):
    if (t + 1) % num_steps_upd == 0 and len(memory_buffer) > MINIBATCH_SIZE:
        return True
    else:
        return False

In [ ]:
# functie care ne da un minibatch de date random din bufferul de actiuni

def get_experiences(memory_buffer):
    experiences = random.sample(memory_buffer, k=MINIBATCH_SIZE)
    states = tf.convert_to_tensor(np.array([e.state for e in experiences if e is not None]), dtype=tf.float32)
    actions = tf.convert_to_tensor(np.array([e.action for e in experiences if e is not None]), dtype=tf.float32)
    rewards = tf.convert_to_tensor(np.array([e.reward for e in experiences if e is not None]), dtype=tf.float32)
    next_states = tf.convert_to_tensor(np.array([e.next_state for e in experiences if e is not None]), dtype=tf.float32)
    done_vals = tf.convert_to_tensor(np.array([e.done for e in experiences if e is not None]).astype(np.uint8), dtype=tf.float32)
    return (states, actions, rewards, next_states, done_vals)

In [ ]:
# functie care ne scade treptat epsilonul

def get_new_eps(epsilon):
    return max(E_MIN, E_DECAY * epsilon)

In [ ]:
# bucla de training

start = time.time()

num_episodes = 2000
max_num_timesteps = 1000

total_point_history = []

num_p_av = 100 # cate experiente sa folosim pentru a vedea media punctajului in timpul trainingului
epsilon = 1.0

# facem bufferul de memorie
memory_buffer = deque(maxlen=MEMORY_SIZE)
experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])

# setam aceleasi valori pentru parametrii si in a doua retea
target_q_network.set_weights(q_network.get_weights())

for i in range(num_episodes):
    
    # resetam simularea si luam state-ul
    state = env.reset()
    state = list(state)[0]
    total_points = 0
    
    for t in range(max_num_timesteps):
        # pentru acest state luam o actiune folosing epsilon-greedy policy
        state_qn = np.expand_dims(state, axis=0)  # modificam dimensiunile state-ului pentru a-l putea trimite in retea
        q_values = q_network(state_qn)
        action = get_action(q_values, epsilon)
        
        # facem actiunea corespunzatoare si luam feedback-ul de la simulator si state-ul urmator
        next_state, reward, done, _, __ = env.step(action)
        next_state = np.array(next_state)
        
        # tinem minte aceasta experienta in memorie
        memory_buffer.append(experience(state, action, reward, next_state, done))
        
        # vedem daca e timpul de un update
        update = check_update_conditions(t, NUM_STEPS_FOR_UPDATE, memory_buffer)
        
        if update:
            # luam un minibatch din buffer
            experiences = get_experiences(memory_buffer)
            
            # facem un pas de gradient descent si dam update la retea
            agent_learn(experiences, GAMMA)
        
        state = next_state.copy()
        total_points += reward
        
        if done:
            break
            
    total_point_history.append(total_points)
    av_latest_points = np.mean(total_point_history[-num_p_av:])
    
    # actualizam epsilon
    epsilon = get_new_eps(epsilon)

    print(f"\rEpisodul {i+1} | Media scorului ultimelor {num_p_av} de episoade: {av_latest_points:.2f}", end="")

    if (i+1) % num_p_av == 0:
        print(f"\rEpisodul {i+1} | Media scorului ultimelor {num_p_av} de episoade: {av_latest_points:.2f}")

    # consideram ca am rezolvat problema daca avem o medie de cel putin 200 de puncte in ultimul timp
    if av_latest_points >= 200.0:
        print(f"\n\nProblema rezolvata in {i+1} episoade de trainig!")
        q_network.save('lunar_lander_model.h5')
        break
        
tot_time = time.time() - start

print(f"\nTimp in care a rulat: {tot_time:.2f} s ({(tot_time/60):.2f} min)")

In [ ]:
# pentru a vedea comportamentul rachetei:

def embed_mp4(filename):
    video = open(filename, "rb").read()
    b64 = base64.b64encode(video)
    tag = """
    <video width="840" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>""".format(
        b64.decode()
    )

    return IPython.display.HTML(tag)


def create_video(filename, env, q_network, fps=30):
    with imageio.get_writer(filename, fps=fps) as video:
        done = False
        state = env.reset()
        state = list(state)[0]
        frame = env.render()
        video.append_data(frame)
        while not done:
            state = np.expand_dims(state, axis=0)
            q_values = q_network(state)
            action = np.argmax(q_values.numpy()[0])
            state, _, done, _, __ = env.step(action)
            frame = env.render()
            video.append_data(frame)

In [ ]:
filename = "lunar_lander.mp4"

create_video(filename, env, q_network)
embed_mp4(filename)